In [15]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
import string

In [2]:
fake=pd.read_csv("fake.csv")
true=pd.read_csv("true.csv")

In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake["Class"]=0
true["Class"]=1

In [8]:
newData=pd.concat([fake,true],axis=0)
newData.sample(6)

,title,text,subject,date,Class
5851,Donald Trump’s Negatives Hit Record Highs Aft...,It looks like Donald Trump is having another r...,News,"June 16, 2016",0
18657,FORMER FBI ASST DIRECTOR: “Jim Comey ‘Danced W...,He threw the reputation of the FBI under the ...,left-news,"May 9, 2017",0
8041,Coolest Mom Ever: Adele Lets Her Son Dress As...,Coming off from her latest record-breaking alb...,News,"February 17, 2016",0
11729,“Your rights are NOT superior to mine!” JUDGE ...,,politics,"Feb 5, 2017",0
23097,BREAKING: At Least 14 US Coalition Military Of...,Syrian Army soldier holds up Al Nusra Front (...,Middle-east,"December 16, 2016",0
14963,Non-secessionist solution to Catalan crisis po...,MADRID (Reuters) - Catalonia s deposed preside...,worldnews,"November 13, 2017",1


In [9]:
newData.drop(["title","subject","date"],axis=1,inplace=True)

In [11]:
newData.reset_index(inplace=True)

In [13]:
newData.drop("index",axis=1,inplace=True)

In [14]:
newData.sample(5)

,text,Class
8051,When Donald Trump launched his presidential ca...,0
32659,(Reuters) - Hewlett Packard Enterprise Co chie...,1
10775,Hillary Clinton just made the claim that she ...,0
19294,Listen here to @legbacarrefour explaining his ...,0
8101,Former Republican Party chairman and U.S. Sena...,0


In [16]:
def clean_text(text):
    text=text.lower()
    text=re.sub("\[.*?\]"," ",text)
    text=re.sub("\\W"," ",text)
    text=re.sub("https?:://\S+/www\.\S+"," ",text)
    text=re.sub("<.*?>+"," ",text)
    text=re.sub("[%s]" %re.escape(string.punctuation)," ",text)
    text=re.sub("\n"," ",text)
    text=re.sub("\w*\d\w*"," ",text)
    return text

In [17]:
newData["text"]=newData["text"].apply(clean_text)

In [20]:
x=newData["text"]
y=newData["Class"]

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [22]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(x_train)
xv_test=vectorizer.transform(x_test)

In [23]:
lg=LogisticRegression()
lg.fit(xv_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [24]:
prediction=lg.predict(xv_test)
prediction

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [25]:
lg.score(xv_test,y_test)

0.9858574610244989

In [26]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4733
           1       0.98      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [27]:
import joblib

In [28]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lg,"logistic.jb")

['logistic.jb']